In [1]:
import pymongo
from pprint import pprint
from pymongo import MongoClient
client = MongoClient("localhost", 27017) #On se connecte a MongoDB

In [2]:
db_agg = client["db_agg"]

In [3]:
website = db_agg["website"]

In [7]:
dico = { "_id" : 1, "domainName" : "test1.com", "hosting" : "hostgator.com" },
{ "_id" : 2, "domainName" : "test2.com", "hosting" : "aws.amazon.com"},
{ "_id" : 3, "domainName" : "test3.com", "hosting" : "aws.amazon.com" },
{ "_id" : 4, "domainName" : "test4.com", "hosting" : "hostgator.com" },
{ "_id" : 5, "domainName" : "test5.com", "hosting" : "aws.amazon.com" },
{ "_id" : 6, "domainName" : "test6.com", "hosting" : "cloud.google.com" },
{ "_id" : 7, "domainName" : "test7.com", "hosting" : "aws.amazon.com" },
{ "_id" : 8, "domainName" : "test8.com", "hosting" : "hostgator.com" },
{ "_id" : 9, "domainName" : "test9.com", "hosting" : "cloud.google.com" },
{ "_id" : 10,"domainName" : "test10.com", "hosting" : "godaddy.com" }

{'_id': 10, 'domainName': 'test10.com', 'hosting': 'godaddy.com'}

In [9]:
import json
x = website.insert_many(dico)

In [14]:
for y in website.find():
    pprint(y)

{'_id': 1, 'domainName': 'test1.com', 'hosting': 'hostgator.com'}


In [17]:
repart_site_hebergeur = { "$group" : {"_id" : "$hosting", "total" : { "$sum" : 1 }} }

In [20]:
tri = {"$sort" : {"total" : -1}}

In [22]:
a = website.aggregate( [repart_site_hebergeur, tri] )

In [31]:
website_grouped = db_agg["website_grouped"]

In [32]:
b =website.aggregate( [repart_site_hebergeur, tri, {"$out" : "website_grouped"}] )

In [39]:
for c in website_grouped.find():
    pprint(c)

{'_id': 'hostgator.com', 'total': 1}


In [55]:
# Q : Combien de sites sont hébergés par amazon ?
x = website.aggregate(
    [{
    "$match" : {"hosting" : "aws.amazon.com"}
    },
    {
    "$group" : { "_id" : "$hosting", "total" : { "$sum" : 1 } }
    }]
  )

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'db_agg'), 'website')


### Exercice  Aggréger des restaurants   

In [2]:
import pymongo
from pprint import pprint
from pymongo import MongoClient
client = MongoClient("localhost", 27017) #On se connecte a MongoDB

In [3]:
restaurants = client["restaurants"]

In [4]:
for x in client.list_database_names():
    pprint(x)

'Argenteuil'
'Test'
'admin'
'config'
'db_agg'
'local'
'metflix'
'movies_artists'
'restaurants'


In [7]:
restaurantC = restaurants["restaurants"]

In [8]:
# Q : La répartition des restaurants par quartier ?

In [42]:
tri = {"$sort":{"_id":1}}
repartition = { "$group" : {"_id" : "$borough", "total" : { "$sum" : 1 }} }

In [1]:
for x in restaurantC.aggregat([tri]):
    pprint(x)

NameError: name 'restaurantC' is not defined

In [52]:
#Le 0 dans grades.0.grade correspond au fait qu'il s'agit de la derniere note, la plus récente

# Q : La répartition des restaurants dont la dernière inspection a donné un grade "C" par quartier ?
note = { "$match" : {"grades.0.grade" : "C" }}
projet = { "$project": {"name": 1,"_id":0,"borough" : 1}}
tri = {"$sort":{"_id":1}}
repartition = { "$group" : {"_id" : "$borough", "total" : { "$sum" : 1 }} }
for x in restaurantC.aggregate([note,tri,repartition]):
    pprint(x)

{'_id': 'Manhattan', 'total': 83}
{'_id': 'Brooklyn', 'total': 56}
{'_id': 'Queens', 'total': 47}
{'_id': 'Staten Island', 'total': 7}
{'_id': 'Bronx', 'total': 27}


In [167]:
# Q : Calculer le score moyen des resto par quartier et trier par score décroissant ?
# Indice : utiliser l'opérateur unwid
#$avg permet d'obtenir une moyenne
Liste_note = {"$unwind":"$grades"}
quartier = { "$group" : {"_id" : "$borough", "moyenne" : { "$avg" : "$grades.score" }} }
Tri = {"$sort":{"moyenne":-1}}
for x in restaurantC.aggregate([Liste_note,quartier,Tri]):
    pprint(x)

{'_id': 'Queens', 'moyenne': 11.634865110930088}
{'_id': 'Brooklyn', 'moyenne': 11.447723132969035}
{'_id': 'Manhattan', 'moyenne': 11.41823125728344}
{'_id': 'Staten Island', 'moyenne': 11.370957711442786}
{'_id': 'Bronx', 'moyenne': 11.036186099942562}
{'_id': 'Missing', 'moyenne': 9.632911392405063}


###                   Exercice  Aggréger des transactions         #####

In [121]:
#Q : Créer une collection "transactions" à partir du fichier transactions.json et répondre aux questions suivantes :

In [56]:
transactions = restaurants["transactions"]

In [55]:
#Q-1. Calculer le montant total des transactions ?
#Liste_transac = {"$unwind":"$Transaction"}
#transac = { "$group" : {"_id" : "$Name","total" : { "$sum" : "$Transaction.price" }} }
#tri = {"$sort":{"_id":1}}
#for x in transactions.aggregate([Liste_transac,transac,tri]):
#    pprint(x)

transac = { "$group" : {"_id" : "total","total" : { "$sum" : "$Payment.Total" }} }
for x in transactions.aggregate([transac]):
    pprint(x)

{'_id': 'total', 'total': 1200}


In [208]:
#Q-2. Calculer le montant total par transaction ?
Liste_transac = {"$unwind":"$Transaction"}
transac = { "$group" : {"_id" : "$Name","total" : { "$sum" : "$Transaction.price" }} }
tri = {"$sort":{"_id":1}}
for x in transactions.aggregate([Liste_transac,transac,tri]):
    pprint(x)

{'_id': 'Dylan', 'total': 410}
{'_id': 'John', 'total': 410}
{'_id': 'Margaret', 'total': 410}
{'_id': 'Tom', 'total': 410}


In [60]:
#Q-3. Calculate total payments (Payment.Total) for each payment type (Payment.Type) ?
Paiement = {"$unwind":"$Payment"}
Ptotale = { "$group" : {"_id" : "$Payment.Type","total" : { "$sum" : "$Payment.Total" }} }
tri = {"$sort":{"_id":1}}
for x in transactions.aggregate([Paiement,Ptotale,tri]):
    pprint(x)
#Pas sur que ce soit bon voir la correction

{'_id': 'Credit-Card', 'total': 800}
{'_id': 'Debit-Card', 'total': 400}


In [61]:
#Q-4. Trouver l'Id le plus élevé.
Idtotale = { "$group" : {"_id" : "$Name","total" : { "$max" : "$Id" }} }
tri = {"$sort":{"_id":1}}
for x in transactions.aggregate([Idtotale,tri]):
    pprint(x)

{'_id': 'Dylan', 'total': 103}
{'_id': 'John', 'total': 100}
{'_id': 'Margaret', 'total': 102}
{'_id': 'Tom', 'total': 101}


In [193]:
#Q-5. Find the max price (Transaction.price) ?
Liste_transac = {"$unwind":"$Transaction"}
transac = { "$group" : {"_id" : "$Name","total" : { "$max" : "$Transaction.price" }} }
tri = {"$sort":{"_id":1}}
for x in transactions.aggregate([Liste_transac,transac,tri]):
    pprint(x)
#Le mieux et d'en afficher un

{'_id': 'Dylan', 'total': 210}
{'_id': 'John', 'total': 210}
{'_id': 'Margaret', 'total': 210}
{'_id': 'Tom', 'total': 210}


## Distinct